In [1]:
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import talib
import torch.utils.data as Data

我想要用lstm的网络来预测收盘价。  
x是价格对于，N日前的收盘价的比例
y是收盘价  

# 获取数据

In [29]:
sys.path.append('../../')
import  DataSource
zz500 = DataSource.get_zz500_codes() # 我看中证500的数据
# 我这里查看第一个股票的吧
dt = DataSource.get_data(zz500[0])

In [30]:
dt.head()

,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST
date,,,,,,,,,,,,,
2010-01-04,sh.600004,5.046509,5.111144,5.041537,5.046509,5.046509,15478854,157747809.0,2,1.525010,1,0.0000,0
2010-01-05,sh.600004,5.046509,5.091256,5.041537,5.066397,5.046509,11070922,112683563.0,2,1.090731,1,0.3941,0
2010-01-06,sh.600004,5.056453,5.066397,4.981874,4.991818,5.066397,9410665,95205863.0,2,0.927159,1,-1.4720,0
2010-01-07,sh.600004,4.971930,4.996790,4.827744,4.862548,4.991818,8961813,88611912.0,2,0.882937,1,-2.5896,0
2010-01-08,sh.600004,4.837688,5.096228,4.837688,5.061425,4.862548,14923812,149969456.0,2,1.470326,1,4.0900,0


In [71]:
N = 60 # 多少日的数据
# 我这里仅仅是记录如下的列的涨跌幅度
rate_columns = ['open', 'high', 'low', 'close']
input_size = len(rate_columns)
seq_len = N    # 多少个时间序列的股票
output_size = 1 # 我只是输出一个今天的收盘价相比较昨日的涨跌比率
num_layers = 10 # 多少个gru合并
hidden_size = 20 # 隐藏层的宽度
batch_size = 16 # 一个批次有多少个数据
lr = 0.01
# 这里先看看怎样快速的截取数据,然后还要归一化，

In [88]:
# 这里只是看对于前日收盘价的比例，这里一次性搞出所有的列
rate_columns_2 = []
dt_list = []
for j in range(N+1):
    # N+1表示要多一天，比如前面30天，我要看看第31天的收盘价相对于第一天的收盘价是什么比例
    for i in rate_columns:
        dt_tmp = pd.DataFrame({f'{i}_rate_{j}':(dt[f'{i}'].shift(-j) - dt[f'preclose'])/dt[f'preclose']})
        dt_list.append(dt_tmp)

dt2 = pd.concat(dt_list[:-(len(rate_columns))], axis=1)
dt2.head()

,open_rate_0,high_rate_0,low_rate_0,close_rate_0,open_rate_1,high_rate_1,low_rate_1,close_rate_1,open_rate_2,high_rate_2,...,low_rate_57,close_rate_57,open_rate_58,high_rate_58,low_rate_58,close_rate_58,open_rate_59,high_rate_59,low_rate_59,close_rate_59
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.000000,0.012808,-0.000985,0.000000,0.000000,0.008867,-0.000985,0.003941,0.001970,0.003941,...,0.115271,0.121182,0.119212,0.135961,0.117241,0.134975,0.133005,0.151724,0.129064,0.150739
2010-01-05,0.000000,0.008867,-0.000985,0.003941,0.001970,0.003941,-0.012808,-0.010837,-0.014778,-0.009852,...,0.117241,0.134975,0.133005,0.151724,0.129064,0.150739,0.151724,0.158621,0.139901,0.157635
2010-01-06,-0.001963,0.000000,-0.016683,-0.014720,-0.018646,-0.013739,-0.047105,-0.040236,-0.045142,0.005888,...,0.124632,0.146222,0.147203,0.154073,0.135427,0.153091,0.151129,0.151129,0.132483,0.141315
2010-01-07,-0.003984,0.000996,-0.032869,-0.025896,-0.030876,0.020916,-0.030876,0.013944,0.021912,0.041833,...,0.152390,0.170319,0.168327,0.168327,0.149402,0.158367,0.157371,0.159363,0.137450,0.139442
2010-01-08,-0.005112,0.048057,-0.005112,0.040900,0.049080,0.069530,0.040900,0.062372,0.056237,0.070552,...,0.179959,0.189162,0.188139,0.190184,0.167689,0.169734,0.173824,0.188139,0.170757,0.186094


In [89]:
dt2.columns

Index(['open_rate_0', 'high_rate_0', 'low_rate_0', 'close_rate_0',
       'open_rate_1', 'high_rate_1', 'low_rate_1', 'close_rate_1',
       'open_rate_2', 'high_rate_2',
       ...
       'low_rate_57', 'close_rate_57', 'open_rate_58', 'high_rate_58',
       'low_rate_58', 'close_rate_58', 'open_rate_59', 'high_rate_59',
       'low_rate_59', 'close_rate_59'],
      dtype='object', length=240)

In [90]:
# 这里转成numpy
x_price = dt2.to_numpy()
x_price = x_price[:-(N+1)]

In [92]:
x_price[0]

array([ 0.        ,  0.01280788, -0.00098522,  0.        ,  0.        ,
        0.008867  , -0.00098522,  0.00394089,  0.00197044,  0.00394089,
       -0.01280788, -0.01083744, -0.01477833, -0.00985222, -0.04334975,
       -0.0364532 , -0.04137931,  0.00985222, -0.04137931,  0.00295567,
        0.01083744,  0.03054187,  0.00295567,  0.02364532,  0.01773399,
        0.03152709,  0.00985222,  0.03054187,  0.01083744,  0.03448276,
        0.00591133,  0.01576355,  0.01477833,  0.05714286,  0.01280788,
        0.05221675,  0.05123153,  0.05714286,  0.03743842,  0.04433498,
        0.04433498,  0.09162562,  0.03448276,  0.07980296,  0.07389163,
        0.10344828,  0.06896552,  0.09359606,  0.0955665 ,  0.09950739,
        0.0729064 ,  0.08275862,  0.10837438,  0.1182266 ,  0.06403941,
        0.0679803 ,  0.07389163,  0.1408867 ,  0.06502463,  0.09162562,
        0.09458128,  0.11133005,  0.04334975,  0.0453202 ,  0.0453202 ,
        0.05714286,  0.00492611,  0.02364532,  0.02463054,  0.03

In [93]:
# 然后进行水平拼接

In [94]:
x_price.shape

(3602, 240)

In [95]:
# 这里需要更改维度
x_shape_old = x_price.shape
x_shape_new_1 = x_shape_old[0]
x_shape_new_3 = len(rate_columns)
x_shape_new_2 = int(x_shape_old[1] / x_shape_new_3)
x_price = x_price.reshape((x_shape_new_1, x_shape_new_2, x_shape_new_3))
x_price.shape

(3602, 60, 4)

In [96]:
dt_list[-(len(rate_columns)- rate_columns.index('close'))].head()

,close_rate_60
date,
2010-01-04,0.157635
2010-01-05,0.145813
2010-01-06,0.122669
2010-01-07,0.155378
2010-01-08,0.218814


In [97]:
dt_list[-(len(rate_columns)- rate_columns.index('close'))].iloc[:-(N+1)]

,close_rate_60
date,
2010-01-04,0.157635
2010-01-05,0.145813
2010-01-06,0.122669
2010-01-07,0.155378
2010-01-08,0.218814
...,...
2024-10-29,-0.041026
2024-10-30,-0.025907
2024-10-31,-0.035015


In [99]:
y_close = dt_list[-(len(rate_columns)- rate_columns.index('close'))][f'close_rate_{N}'].to_numpy()[:-(N+1)]
y_close.shape

(3602,)

In [100]:
y_close[0]

0.15763546798029565

In [101]:
# 裁剪，成整数批次
_x_y_len = int(x_price.shape[0]/batch_size) * batch_size

x_price = x_price[-_x_y_len:]
y_close = y_close[-_x_y_len:]
x_price = x_price.reshape(x_price.shape[0]//batch_size, batch_size, x_price.shape[1], x_price.shape[2])
y_close = y_close.reshape(y_close.shape[0]//batch_size, batch_size, 1)
print(x_price.shape)
print(y_close.shape)

(225, 16, 60, 4)
(225, 16, 1)


In [102]:
# 这里要区分训练集和测试集



dataset = Data.TensorDataset(torch.FloatTensor(x_price), torch.FloatTensor(y_close))
batch_size = 16
train_loader, test_loader = Data.random_split(dataset,
                                             lengths=[int(0.9*len(dataset)), len(dataset)- int(0.9*len(dataset))],
                                             generator=torch.Generator().manual_seed(0))

# 神经网络

In [103]:
# 这里做一个网络
import torch.nn as nn
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.gru(x) # gru层
        out = self.fc(out[:,-1, :]) # 全连接层
        return out

# 训练

In [104]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = GRUModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = torch.nn.MSELoss() # 均方差误差函数
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [105]:
for epoch in range(10):  # 迭代一次
    running_loss = 0.0
    running_loss_2 = []
    for i, data in enumerate(train_loader, 0):
        # 获取输入
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 梯度置 0
        optimizer.zero_grad()
        # 正向传播，反向传播，优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # 打印状态信息
        # running_loss += loss.item()
        running_loss_2.append(loss.item())
        if i % 200 == 199:    # 每 200 批次打印一次
            print(f'[{epoch + 1}, {i + 1}]: loss : max:{max(running_loss_2):.5f}, min:{min(running_loss_2):.5f}, avg:{sum(running_loss_2)/len(running_loss_2):.5f}')
            # print('[%d, %5d] loss: ,max : %.3f,  min: ' , %
            #       (epoch + 1, i + 1, running_loss / 200))
            # running_loss = 0.0
            running_loss_2 = []
    # # 每轮迭代打印一次
    # if epoch % 10 == 0:
    #     print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, loss.item()))
    #     running_loss = 0.0

    
print('Finished Training.')

[1, 200]: loss : max:0.28708, min:0.00019, avg:0.02139
[2, 200]: loss : max:0.11423, min:0.00011, avg:0.00577
[3, 200]: loss : max:0.06379, min:0.00011, avg:0.00356
[4, 200]: loss : max:0.04108, min:0.00008, avg:0.00266
[5, 200]: loss : max:0.02857, min:0.00007, avg:0.00211
[6, 200]: loss : max:0.02182, min:0.00010, avg:0.00180
[7, 200]: loss : max:0.01853, min:0.00008, avg:0.00164
[8, 200]: loss : max:0.01691, min:0.00007, avg:0.00157
[9, 200]: loss : max:0.01589, min:0.00006, avg:0.00153
[10, 200]: loss : max:0.01501, min:0.00006, avg:0.00148
Finished Training.


# 测试数据

In [107]:
with torch.no_grad():
    running_loss_2 = []
    for i, data in enumerate(test_loader, 0):
        # 获取输入
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 正向传播，反向传播，优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        running_loss_2.append(loss.item())
    # 每一次显示一下损失的最大值和最小值
    print(f'loss: max: {max(running_loss_2)}, min:{min(running_loss_2)}, avg:{sum(running_loss_2)/len(running_loss_2)}')

10, loss: max: 0.011340830475091934, min:0.0008061322150751948, avg:0.0030817871650114007


# 保存模型

In [109]:
import os
model_path = os.path.join('./模型', f"{zz500[0]}.pth")
torch.save(net, model_path)

# 读取数据的函数

In [110]:
def get_loader(code_name, N=60, rate_columns=['open', 'high', 'low', 'close'], batch_size=16, train_rate=0.9):
    dt = DataSource.get_data(code_name)
    dt_list = []
    for j in range(N+1):
        # N+1表示要多一天，比如前面30天，我要看看第31天的收盘价相对于第一天的收盘价是什么比例
        for i in rate_columns:
            dt_tmp = pd.DataFrame({f'{i}_rate_{j}':(dt[f'{i}'].shift(-j) - dt[f'preclose'])/dt[f'preclose']})
            dt_list.append(dt_tmp)
    
    dt2 = pd.concat(dt_list[:-(len(rate_columns))], axis=1)
    # x
    x_price = dt2.to_numpy()
    x_price = x_price[:-(N+1)]
    # 更改维度
    x_shape_old = x_price.shape
    x_shape_new_1 = x_shape_old[0]
    x_shape_new_3 = len(rate_columns)
    x_shape_new_2 = int(x_shape_old[1] / x_shape_new_3)
    x_price = x_price.reshape((x_shape_new_1, x_shape_new_2, x_shape_new_3))
    # y
    y_close = dt_list[-(len(rate_columns)- rate_columns.index('close'))][f'close_rate_{N}'].to_numpy()[:-(N+1)]
    # 然后是批次的整数倍
    _x_y_len = int(x_price.shape[0]/batch_size) * batch_size
    
    x_price = x_price[-_x_y_len:]
    y_close = y_close[-_x_y_len:]
    x_price = x_price.reshape(x_price.shape[0]//batch_size, batch_size, x_price.shape[1], x_price.shape[2])
    y_close = y_close.reshape(y_close.shape[0]//batch_size, batch_size, 1)
    # 做成加载器
    dataset = Data.TensorDataset(torch.FloatTensor(x_price), torch.FloatTensor(y_close))
    train_loader, test_loader = Data.random_split(dataset,
                                                 lengths=[int(train_rate*len(dataset)), len(dataset)- int(train_rate*len(dataset))],
                                                 generator=torch.Generator().manual_seed(0))
    return train_loader, test_loader

    

In [115]:
for i  in range(len(zz500)):
    train_loader2, test_loader2 = get_loader(zz500[i], train_rate=0.1)
    # 我这里用测试集来测试
    running_loss_2 = []
    for i, data in enumerate(test_loader2, 0):
            # 获取输入
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # 正向传播，反向传播，优化
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            running_loss_2.append(loss.item())
    # 每一次显示一下损失的最大值和最小值
    print(f'{i}:{code_name}：loss: max: {max(running_loss_2)}, min:{min(running_loss_2)}, avg:{sum(running_loss_2)/len(running_loss_2)}')


sh.600004：loss: max: 0.05766661837697029, min:0.00013507597032003105, avg:0.0031200260577122383
sh.600007：loss: max: 0.11735248565673828, min:0.00029643840389326215, avg:0.003640793509072546
sh.600008：loss: max: 0.08081412315368652, min:0.0005158991552889347, avg:0.004678237290073443
sh.600021：loss: max: 0.8158788681030273, min:0.00021617321181111038, avg:0.017153640181196653
sh.600032：loss: max: 0.03822392225265503, min:0.00011160094436490908, avg:0.003452598753396392
sh.600038：loss: max: 0.045919544994831085, min:0.00048359096399508417, avg:0.003588942383797931
sh.600056：loss: max: 1.3841007947921753, min:0.00019056754535995424, avg:0.014785106859302946
sh.600060：loss: max: 0.20330774784088135, min:0.00012914305261801928, avg:0.0071659169683555195
sh.600062：loss: max: 0.7862763404846191, min:0.0002741701027844101, avg:0.007390127295204966
sh.600079：loss: max: 0.10253335535526276, min:0.0003211036091670394, avg:0.004253290920791313
sh.600095：loss: max: 0.21937155723571777, min:0.00025